In [4]:
import { Ollama } from "@langchain/community/llms/ollama";
import { HumanMessage } from "@langchain/core/messages";
import { StringOutputParser } from "@langchain/core/output_parsers";

const ollama = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "llama3", 
});

const outputPrase = new StringOutputParser();

const simpleChain = ollama.pipe(outputPrase)

const stream = await simpleChain.invoke([
     new HumanMessage("讲个故事")
])

console.log(stream);

// const stream = await simpleChain.stream([
//      new HumanMessage("Tell me a joke")
// ])
// for await (const chunk of stream){
//     console.log(chunk)
// }

Let me tell you a story.

Once upon a time, in a small village nestled in the mountains, there lived a young girl named Ling. She was known throughout the village for her kind heart and her love of animals.

One winter morning, Ling woke up to find that a big storm had rolled in overnight. The wind howled and the snow fell in thick, heavy flakes. Ling looked out the window and saw that the villagers were struggling to make their way through the drifts.

Without hesitation, Ling grabbed her warmest coat and set out into the storm. She trudged through the snow, her eyes scanning the ground for any sign of life. As she walked, she noticed a small bird perched on a branch, shivering in the cold.

Ling's heart went out to the little creature. She knew that if she didn't help it soon, the bird might not make it through the night. So, she carefully scooped up some snow and molded it into a warm ball. Then, she gently placed the bird on top of the ball, making sure it was cozy and safe.

As sh

In [7]:
import { PromptTemplate } from "@langchain/core/prompts";

const greetingPrompt = new PromptTemplate({
  inputVariables: [],
  template: "hello world",
});
const formattedGreetingPrompt = await greetingPrompt.format();

console.log(formattedGreetingPrompt);

hello world


In [8]:
const multiVariableGreetingPrompt = new PromptTemplate({
  inputVariables: ["timeOfDay", "name"],
  template: "good {timeOfDay}, {name} {{test}}",
});
const formattedMultiVariableGreeting = await multiVariableGreetingPrompt.format({
  timeOfDay: "morning",
  name: "Kai",
});

console.log(formattedMultiVariableGreeting);

good morning, Kai {test}


In [10]:
const autoInferTemplate = PromptTemplate.fromTemplate("good {timeOfDay}, {name}");
console.log(autoInferTemplate.inputVariables);

const formattedAutoInferTemplate = await autoInferTemplate.format({
  timeOfDay: "morning",
  name: "Kai",
});
console.log(formattedAutoInferTemplate)

[ "timeOfDay", "name" ]
good morning, Kai


In [11]:
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

const promptWithDate = new PromptTemplate({
  template: "今天是{date}，{activity}。",
  inputVariables: ["date", "activity"],
});

const partialedPromptWithDate = await promptWithDate.partial({
  date: getCurrentDateStr,
});

const formattedPromptWithDate = await partialedPromptWithDate.format({
  activity: "我们去爬山",
});

console.log(formattedPromptWithDate);

今天是2024/5/9，我们去爬山。


In [12]:
const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

function generateGreeting(timeOfDay) {
  return () => {
    const date = getCurrentDateStr()
    switch (timeOfDay) {
      case 'morning':
        return date + ' 早上好';
      case 'afternoon':
        return date + ' 下午好';
      case 'evening':
        return date + ' 晚上好';
      default:
        return date + ' 你好';
    }
  };
}

const prompt = new PromptTemplate({
  template: "{greeting}!",
  inputVariables: ["greeting"],
});

const currentTimeOfDay = 'afternoon';
const partialPrompt = await prompt.partial({
  greeting: generateGreeting(currentTimeOfDay),
});

const formattedPrompt = await partialPrompt.format();

console.log(formattedPrompt);

2024/5/9 下午好!


In [13]:
import { SystemMessagePromptTemplate } from "@langchain/core/prompts";

const translateInstructionTemplate = SystemMessagePromptTemplate.fromTemplate(`你是一个专
业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。`);

In [14]:
import { HumanMessagePromptTemplate } from "@langchain/core/prompts";

const userQuestionTemplate = HumanMessagePromptTemplate.fromTemplate("请翻译这句话：{text}")

In [15]:
import { ChatPromptTemplate } from "@langchain/core/prompts";

const chatPrompt = ChatPromptTemplate.fromMessages([
  translateInstructionTemplate,
  userQuestionTemplate,
]);

In [16]:
const formattedChatPrompt = await chatPrompt.formatMessages({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
});

console.log(formattedChatPrompt)

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "你是一个专\n业的翻译员，你的任务是将文本从中文翻译成法语。",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "你是一个专\n业的翻译员，你的任务是将文本从中文翻译成法语。",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "请翻译这句话：你好，世界",
      additional_kwargs: {},
      response_metadata: {}
    },
    lc_namespace: [ "langchain_core", "messages" ],
    content: "请翻译这句话：你好，世界",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {}
  }
]


In [18]:
const systemTemplate = "你是一个专业的翻译员，你的任务是将文本从{source_lang}翻译成{target_lang}。";
const humanTemplate = "请翻译这句话：{text}";

const chatPrompt = ChatPromptTemplate.fromMessages([
  ["system", systemTemplate],
  ["human", humanTemplate],
]);

In [20]:
import { load } from "dotenv";
import { Ollama } from "@langchain/community/llms/ollama";
import { StringOutputParser } from "@langchain/core/output_parsers";
import { HumanMessage } from "@langchain/core/messages";

const ollama = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "llama3", 
});

const env = await load();
const process = {
    env
}

const outputPraser = new StringOutputParser();

const chain = chatPrompt.pipe(ollama).pipe(outputPraser);

const stream = await chain.invoke({
  source_lang: "中文",
  target_lang: "法语",
  text: "你好，世界",
})

console.log(stream);

La traduction est :

Bonjour, monde !

(Note : 在法语中，常见的问候语为"bonjour"，表示早上或下午的问候，而不是特别关心对方的情况。但是，如果你想表达更友好的意思，可以使用"salut"或"hello"。)

如果你想要更多的翻译，请随时提出请求！


In [21]:
import {
  PromptTemplate,
  PipelinePromptTemplate,
} from "@langchain/core/prompts";

const getCurrentDateStr = () => {
  return new Date().toLocaleDateString();
};

const fullPrompt = PromptTemplate.fromTemplate(`
你是一个智能管家，今天是 {date}，你的主人的信息是{info}, 
根据上下文，完成主人的需求
{task}`);

const datePrompt = PromptTemplate.fromTemplate("{date}，现在是 {period}")
const periodPrompt = await datePrompt.partial({
    date: getCurrentDateStr
})

const infoPrompt =  PromptTemplate.fromTemplate("姓名是 {name}, 性别是 {gender}");

const taskPrompt = PromptTemplate.fromTemplate(`
我想吃 {period} 的 {food}。 
再重复一遍我的信息 {info}`);

const composedPrompt = new PipelinePromptTemplate({
  pipelinePrompts: [
    {
      name: "date",
      prompt: periodPrompt,
    },
    {
      name: "info",
      prompt: infoPrompt,
    },
    {
      name: "task",
      prompt: taskPrompt,
    },
  ],
  finalPrompt: fullPrompt,
});

const formattedPrompt = await composedPrompt.format({
    period: "早上",
    name: "张三",
    gender: "male",
    food: "lemon"
});

console.log(formattedPrompt)


你是一个智能管家，今天是 2024/5/9，现在是 早上，你的主人的信息是姓名是 张三, 性别是 male, 
根据上下文，完成主人的需求

我想吃 早上 的 lemon。 
再重复一遍我的信息 姓名是 张三, 性别是 male


In [22]:
import { StructuredOutputParser } from "langchain/output_parsers";
import { PromptTemplate } from "@langchain/core/prompts";

const parser = StructuredOutputParser.fromNamesAndDescriptions({
  answer: "用户问题的答案",
  evidence: "你回答用户问题所依据的答案",
  confidence: "问题答案的可信度评分，格式是百分数",
});

In [23]:
import { Ollama } from "@langchain/community/llms/ollama";

const prompt = PromptTemplate.fromTemplate("尽可能的回答用的问题 \n{instructions} \n{question}")

const ollama = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "llama3", 
});

const chain = prompt.pipe(ollama).pipe(parser)

const res = await chain.invoke({
    question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
    instructions: parser.getFormatInstructions()
})
                               
console.log(res)

{
  answer: "Leonardo da Vinci",
  evidence: "Leonardo da Vinci was an Italian polymath and one of the most famous artists in history. He is widel"... 60 more characters,
  confidence: "99%"
}


In [24]:
import { CommaSeparatedListOutputParser } from "@langchain/core/output_parsers";

const parser = new CommaSeparatedListOutputParser();

console.log(parser.getFormatInstructions())

Your response should be a list of comma separated values, eg: `foo, bar, baz`


In [26]:
const prompt = PromptTemplate.fromTemplate("列出3个 {country} 的著名的互联网公司.\n{instructions}")

const ollama = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "llama3", 
});
    
const chain = prompt.pipe(ollama).pipe(parser)

const response = await chain.invoke({
    country: "America",
    instructions: parser.getFormatInstructions(),
});
console.log(response)

[ "Google", "Facebook", "Amazon" ]


In [27]:
import { z } from "zod";

const schema = z.object({
  answer:  z.string().describe("用户问题的答案"),
  confidence: z.number().min(0).max(100).describe("问题答案的可信度评分，满分 100")
});

In [28]:
const parser = StructuredOutputParser.fromZodSchema(schema);
const prompt = PromptTemplate.fromTemplate("尽可能的回答用的问题 \n{instructions} \n{question}")
const ollama = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "llama3", 
});

const chain = prompt.pipe(ollama).pipe(parser)
const res = await chain.invoke({
    question: "蒙娜丽莎的作者是谁？是什么时候绘制的",
    instructions: parser.getFormatInstructions()
})
                               
console.log(res)

{ answer: "Leonardo da Vinci", confidence: 95 }


In [2]:
import { OutputFixingParser } from "langchain/output_parsers";

const ollama = new Ollama({
  baseUrl: "http://localhost:11434", 
  model: "llama3", 
});

const fixParser = OutputFixingParser.fromLLM(ollama, parser);

ReferenceError: Ollama is not defined

In [1]:
const wrongOutput = {
  "answer": "蒙娜丽莎的作者是达芬奇，大约在16世纪初期（1503年至1506年之间）开始绘制。",
  "sources": "90%" 
};

const fixParser = OutputFixingParser.fromLLM(ollama, parser);
const output = await fixParser.parse(JSON.stringify(wrongOutput));
console.log(output)

ReferenceError: OutputFixingParser is not defined